In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# Load dataset
data = pd.read_csv("/content/IMDB Dataset.csv")

# Display the first few rows and the shape of the dataset
print(data.head())
print(data.shape)


                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive
(50000, 2)


In [ ]:
# Display the last few rows of the dataset
print(data.tail())

# Count the sentiment values
print(data["sentiment"].value_counts())

# Map sentiment values to numerical format
data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)

# Display the first and last few rows after replacing sentiment values
print(data.head())
print(data.tail())

# Verify sentiment value counts
print(data["sentiment"].value_counts())


                                                  review sentiment
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative
sentiment
positive    25000
negative    25000
Name: count, dtype: int64
                                              review  sentiment
0  One of the other reviewers has mentioned that ...          1
1  A wonderful little production. <br /><br />The...          1
2  I thought this was a wonderful way to spend ti...          1
3  Basically there's a family where a little boy ...          0
4  Petter Mattei's "Love in the Time of Money" is...          1
                                                  review  sentiment
49995  I thought this movie did a down right good job...          1
49996 

In [ ]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Print the shape of training and testing sets
print(train_data.shape)
print(test_data.shape)


(40000, 2)
(10000, 2)


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenize the text data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])

# Pad sequences
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

# Assign sentiment labels
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM

# Build the LSTM model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))

# Print the model summary
model.summary()

# Compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 128)          640000    
                                                                 
 lstm (LSTM)                 (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 771713 (2.94 MB)
Trainable params: 771713 (2.94 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Train the model
model.fit(X_train, Y_train, epochs=5, batch_size=64, validation_split=0.2)

# Save the model and tokenizer
model.save("model.h5")
import joblib
joblib.dump(tokenizer, "tokenizer.pkl")


Epoch 1/5
500/500 [==============================] - 357s 707ms/step - loss: 0.4209 - accuracy: 0.8007 - val_loss: 0.3411 - val_accuracy: 0.8566
Epoch 2/5
500/500 [==============================] - 353s 706ms/step - loss: 0.2733 - accuracy: 0.8885 - val_loss: 0.2947 - val_accuracy: 0.8786
Epoch 3/5
500/500 [==============================] - 361s 721ms/step - loss: 0.2201 - accuracy: 0.9139 - val_loss: 0.2810 - val_accuracy: 0.8811
Epoch 4/5
500/500 [==============================] - 358s 717ms/step - loss: 0.1819 - accuracy: 0.9289 - val_loss: 0.3080 - val_accuracy: 0.8685
Epoch 5/5
500/500 [==============================] - 355s 711ms/step - loss: 0.1674 - accuracy: 0.9360 - val_loss: 0.3842 - val_accuracy: 0.8720


['tokenizer.pkl']

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")


313/313 [==============================] - 30s 91ms/step - loss: 0.3644 - accuracy: 0.8779
Loss: 0.36436212062835693
Accuracy: 0.8779000043869019


In [ ]:
# Define the predictive system function
def predictive_system(review):
    sequences = tokenizer.texts_to_sequences([review])
    padded_sequence = pad_sequences(sequences, maxlen=200)
    prediction = model.predict(padded_sequence)
    sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
    return sentiment

# Test the predictive system
print(predictive_system("This movie was fantastic and amazing"))
print(predictive_system("A thrilling adventure with stunning visuals"))
print(predictive_system("A visual masterpiece"))


1/1 [==============================] - 1s 540ms/step
positive
1/1 [==============================] - 0s 84ms/step
positive
1/1 [==============================] - 0s 88ms/step
positive


In [ ]:
from keras.models import load_model
import joblib

# Reload the model and tokenizer
model = load_model("/content/model.h5")
tokenizer = joblib.load("/content/tokenizer.pkl")

# Redefine the predictive system function
def predictive_system(review):
    sequences = tokenizer.texts_to_sequences([review])
    padded_sequence = pad_sequences(sequences, maxlen=200)
    prediction = model.predict(padded_sequence)
    sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
    return sentiment

# Test the reloaded predictive system
review_sentiment = predictive_system("Beautiful cinematography")
print(review_sentiment)


1/1 [==============================] - 0s 302ms/step
positive


In [ ]:
# Install Gradio for creating a web interface
!pip install gradio

# Create a Gradio interface for the predictive system
import gradio as gr
title = "MOVIE SENTIMENT ANALYSIS APPLICATION"
app = gr.Interface(fn=predictive_system, inputs="textbox", outputs="textbox", title=title)

# Launch the Gradio app
app.launch(share=True)


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.2/318.2 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━